In [1]:
import json
from time import sleep
from flask import Flask
from normalizer import *
from kafka import KafkaConsumer, KafkaProducer
from langdetect import detect




def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        
        producer_instance.send(topic_name, key=key_bytes.lower(), value=value_bytes.lower())
        producer_instance.flush()
        print('Corpus published successfully.')
    except Exception as ex:
        print('Exception in publishing the Corpus')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer


def parse(tweet):
    title = str(tweet)
    rec = {}
    try:
        rec = {'tweet': title}

    except Exception as ex:
        print('Exception while parsing')
        print(str(ex))
    finally:
        return json.dumps(rec)
    

def text_generation(lisTweet):
    return '. '.join(lisTweet)
    

if __name__ == '__main__':
    print("===> Generating the corpus ...")
    print()
    reords_corpus=[]
    list_tweet=[]
    
    topic_name = 'normalized_tweet' 
    parsed_topic_name = 'corpus_tweet'
    
    consumer = KafkaConsumer(topic_name, auto_offset_reset='earliest',
                             bootstrap_servers=['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000)
    
    for msg in consumer:
        record=json.loads(msg.value)
        text=record['tweet']
        
        try: 
            if detect(text)=='en':
                doc = nlp(text)
                list_tweet.append(text)
        except:
            pass
        #print(doc)
        sleep(3)
    #text=text_generation(list_tweet)
    for text in list_tweet:
        print(text)
    
    #reords_corpus.append(parse(text.strip()))
    
    if len(reords_corpus) > 0:
        producer = connect_kafka_producer()
        for rec in reords_corpus:
            publish_message(producer, parsed_topic_name, 'corpus', rec)
    

===> Generating the corpus ...

sat in hosp while tom sleep he fed well and seems be recovering fine take that meningitis
woo meningitis vaccine i love shot
meningitis warning after family scare
did nt feel good said i had all the symptom of meningitis so they had run all these test but i do nt have it
same symptom of meningitis encephalitis they are always the same could be
i anyone who can spare a pound or i am running for meningitis research for little leo pls rd
charity walk in memory of popular teenager who died from meningitis friend of popular teenager jodie willan are
that time of year mile great midland fun run for the meningitis trust
group calls for meningitis booster in school
meningitis kansa state university researcher awarded nih grant study antibiotic resistant anti
urggk someone at work got us damn student with their hope it is not meningitis
running for the meningitis trust next call bupa london km please donate if you
i really fucking hope i do nt have meningitis
try